In [6]:
import numpy as np
from math import log

In [80]:
# for the leaf node in which has impurity, we determine the class based on the majority of classes
def majorityCount(classList):
    labelKeys, labelCounts = np.unique(classList, return_counts=True)
    majorityClass = labelKeys[np.argsort(labelCounts)[-1]]
    return majorityClass


def calcEntropy(datasets):
    numEntries = len(datasets)
    """
    labels = datasets[:, -1]
    labelKeys, labelCounts = np.unique(labels, return_counts=True)
    entropy = 0
    for key, count in zip(labelKeys, labelCounts):
        prob = count / numEntries
        entropy += -prob * log(prob, 2)
    """
    labelCounts = {}
    for featVect in datasets: #the the number of unique elements and their occurance
        currentLabel = str(featVect[-1])
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0 # construct a new count for this label key
        labelCounts[currentLabel] += 1
    
    entropy = 0
    for key in labelCounts.keys():
        prob = labelCounts[key] / numEntries
        entropy += - prob * log(prob, 2)
    return entropy

# splitting each attribute(column) based on different features:瓜的色泽是青绿乌黑还是浅白
def splitDataSet(datasets, featureAxis, featureValue):
    """
    Description:split a single feature based on its different feature values;
    Inputs:
    data:input data in a certain node
    featureAxis: Nr. i.th feature/attribute of data
    featureValue: all the categorical values in this feature
    
    return: splitted subdatasets but pulls away this feature dimension,
            convinent for next tree split.
    """
    remainData = []
    for entry in datasets:
        if entry[featureAxis] == featureValue:
            reducedVector = entry[:featureAxis]
            reducedVector.extend(entry[featureAxis + 1:])
            remainData.append(reducedVector)
    return remainData
    

# select the "best" feature to split
def bestFeatureToSplit(datasets):
    numFeatures = len(datasets[0]) - 1  # number of features
    baseEntropy = calcEntropy(datasets)
    bestInfoGain = 0
    bestFeatureIndex = 0
    for i in range(numFeatures):
        newEntropy = 0
        featureVector = [data[i] for data in datasets]
        featureSets = np.unique(featureVector)
        for featureValue in featureSets:
            remainData = splitDataSet(datasets, i, featureValue)
            prob = len(remainData) / float(len(datasets))
            newEntropy += - prob * calcEntropy(remainData)
        reducedEntropy = baseEntropy - newEntropy  # reduction od impuroty
        if reducedEntropy > bestInfoGain:
            bestInfoGain = reducedEntropy
            bestFeatureIndex = i
    return bestFeatureIndex

def createTree(datasets, featureSets):
    """
    datasets: data colums plus a colum of labels
    featureSets: a set including all names of features/attributes
    """
    classLabels = [data[-1] for data in datasets]
    if len(datasets[0]) == 1:  # after traversing all of attributes
        return majorityCount(classLabels)
    elif len(classLabels[0]) == len(np.unique(classLabels)): # all the elements in this node belong to the same class
        return classLabels[0]
    else:
        bestFeature = bestFeatureToSplit(datasets)  # best splitting feature
        bestFeatLabel = featureSets[bestFeature]  # String variable
        del(featureSets[bestFeature]) # sub feature sets excluding the former best feature name
        
        myTree = {bestFeatLabel: {}}  # 嵌套字典
        featureVector = [data[bestFeature] for data in datasets]
        uniqueValue = np.unique(featureVector)
        for value in uniqueValue:
            subFeatureSets = featureSets[:]
            remainData = splitDataSet(datasets, bestFeature, value)
            myTree[bestFeatLabel][value] = createTree(remainData, subFeatureSets)
        return myTree        
    
def classify(inputTree, featureSets, testVector):
    """
    Input a test vector and predict the class label
    """
    firstSplitStr = list(inputTree.keys())[0]  # extract the outermost layer of key
    index = featureSets.index(firstSplitStr)  # this string keys refers to which index in featureSets
    featureKey = testVector[index]
    secondDict = inputTree[firstSplitStr]
    valueOfFeatKey = secondDict[featureKey]
    if isinstance(valueOfFeatKey, dict): # whether valueOfFeatKey is a dictionary
        classLabel = classify(valueOfFeatKey, featureSets, testVector)
    else:
        classLabel = valueOfFeatKey
    return classLabel

In [81]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values                             
    return dataSet, labels
myDat, labels = createDataSet()
myTree = createTree(myDat, labels)
print(myTree)

{'flippers': {0: {'no surfacing': {1: 'no'}}, 1: {'no surfacing': {0: 'no', 1: 'yes'}}}}


In [82]:
myDat, labels = createDataSet()
classLabel = classify(myTree, labels, [1,1])
print(classLabel)

yes


In [87]:
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']

# open the file
file = open("lenses.txt")
arrayOfLines = file.readlines()
datasets = []
for line in arrayOfLines:
    line = line.strip()
    listFromLine = line.split('\t')
    datasets.append(listFromLine)
datasets

[['young', 'myope', 'no', 'reduced', 'no lenses'],
 ['young', 'myope', 'no', 'normal', 'soft'],
 ['young', 'myope', 'yes', 'reduced', 'no lenses'],
 ['young', 'myope', 'yes', 'normal', 'hard'],
 ['young', 'hyper', 'no', 'reduced', 'no lenses'],
 ['young', 'hyper', 'no', 'normal', 'soft'],
 ['young', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['young', 'hyper', 'yes', 'normal', 'hard'],
 ['pre', 'myope', 'no', 'reduced', 'no lenses'],
 ['pre', 'myope', 'no', 'normal', 'soft'],
 ['pre', 'myope', 'yes', 'reduced', 'no lenses'],
 ['pre', 'myope', 'yes', 'normal', 'hard'],
 ['pre', 'hyper', 'no', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'no', 'normal', 'soft'],
 ['pre', 'hyper', 'yes', 'reduced', 'no lenses'],
 ['pre', 'hyper', 'yes', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'no', 'normal', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'reduced', 'no lenses'],
 ['presbyopic', 'myope', 'yes', 'normal', 'hard'],
 ['presbyopic', 

In [90]:
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']
trainingTree = createTree(datasets, lensesLabels)
trainingTree

{'age': {'pre': {'prescript': {'hyper': {'astigmatic': {'no': {'tearRate': {'normal': 'soft',
        'reduced': 'no lenses'}},
      'yes': {'tearRate': {'normal': 'no lenses', 'reduced': 'no lenses'}}}},
    'myope': {'astigmatic': {'no': {'tearRate': {'normal': 'soft',
        'reduced': 'no lenses'}},
      'yes': {'tearRate': {'normal': 'hard', 'reduced': 'no lenses'}}}}}},
  'presbyopic': {'prescript': {'hyper': {'astigmatic': {'no': {'tearRate': {'normal': 'soft',
        'reduced': 'no lenses'}},
      'yes': {'tearRate': {'normal': 'no lenses', 'reduced': 'no lenses'}}}},
    'myope': {'astigmatic': {'no': {'tearRate': {'normal': 'no lenses',
        'reduced': 'no lenses'}},
      'yes': {'tearRate': {'normal': 'hard', 'reduced': 'no lenses'}}}}}},
  'young': {'prescript': {'hyper': {'astigmatic': {'no': {'tearRate': {'normal': 'soft',
        'reduced': 'no lenses'}},
      'yes': {'tearRate': {'normal': 'hard', 'reduced': 'no lenses'}}}},
    'myope': {'astigmatic': {'no': 

In [91]:
lensesLabels = ['age', 'prescript', 'astigmatic', 'tearRate']
testVector = ['young', 'myope', 'yes', 'normal']
classLabel = classify(trainingTree, lensesLabels, testVector)
classLabel

'hard'